In [1]:
import requests as req
from requests import get
from bs4 import BeautifulSoup as bf
import openpyxl
# from openpyxl.drwing.image import Image 
import pandas as pd
import os
from datetime import datetime, date
import datetime 
import time
import re
from IPython.display import Image
import json
import io
import jpype
import asposecells
from urllib.parse import urlparse, urljoin


KeyboardInterrupt



In [ ]:
URLs = [ 
        'https://www.ettoday.net/news/focus/%E6%94%BF%E6%B2%BB/',  #政治
        'https://www.ettoday.net/news/focus/%E7%A4%BE%E6%9C%83/',   #社會
        'https://www.ettoday.net/news/focus/%E5%A4%A7%E9%99%B8/',   #大陸   時間會有問題
        'https://www.ettoday.net/news/focus/%E7%94%9F%E6%B4%BB/',  #生活新聞
       ]

df_rows = []

In [ ]:
for URL in URLs:
    request = req.get(URL)
    sp = bf(request.text, 'html.parser')
    sel = sp.find_all('div', {'class': 'piece clearfix'})[:20]
#     category = sp.find('em', class_='icon').find_next_sibling('span').text
    try:
        category = sp.find('em', class_='icon').find_next_sibling('span').text
    except AttributeError:
        category = sp.find('h2', class_='block_title').text.strip()




    for i in sel:
        link = i.find('a').get('href')    # 各個的連結   
        if not urlparse(link).scheme:
            link = urljoin(URL, link)
        
        Link_Content = req.get(link)
        sp_inner = bf(Link_Content.text, 'html.parser')
        
#         index = sp_inner.find('strong')
        header = sp_inner.find('h1', class_='title')
        publish_time = sp_inner.find('time').text
        Inner_contents = [p.text for p in sp_inner.find_all('p')[3:] if not p.find('strong')]
        end_index = next((i for i, p in enumerate(Inner_contents) if 
                          "分享給朋友：" in p), None)
       
        if header:  
            header = header.get_text(strip=True)  # 看標標題有沒有
        else:
            print("No header found on page:", link)
            
        if end_index is not None:
            Inner_contents = Inner_contents[:end_index]     

#         Contents = []   # 看有沒有其他方法
        
        print(category)
        print(header)
        print(link)
        print(publish_time)
        
        for Inner_content in Inner_contents:  
            print(Inner_content)
#             Contents.append(Inner_content)
        df_rows.append({
                    'Category': category,  
                    'Title': header,
                    'Link': link,
                    'Publish time': publish_time,
#                     'Content': Contents
                     'Content': '\n'.join(Inner_contents)
                    }
                        )

In [ ]:
df = pd.DataFrame(df_rows)


today = datetime.date.today().strftime('%Y-%m-%d')
file_name = 'ET Today' + today 
file_path = 'C:/Users/Administrator/INTERN/testing'

df.to_excel(os.path.join(file_path, file_name + '.xlsx'), 
            index=False, encoding='UTF-8-Sig', engine='openpyxl')